# Speed of symbolic computation of stability polynomials

Symbolic computation of stability polynomials and internal stability polynomials in SymPy can get very slow for methods with many stages.  The stability polynomial can be computed in a variety of ways, and different ways seem to work better for different classes of RK methdods.  Let's investigate.

In [ ]:
%pylab inline
import sympy
from nodepy import rk
import time
import matplotlib.pyplot as plt
import numpy as np

Next we have 3 functions for computing the stability polynomial in different ways.  The first uses the matrix-inverse formula, the second uses the ratio of determinants, and the third uses a power series for the matrix inverse.  They are implemented for both the Shu-Osher and Butcher forms.

Note that the ratio of determinants in the Shu-Osher form is still not implemented correcly.

Also note that for implicit methods the ratio of determinants is the preferred method, since it gives formulas for the numerator and denominator.  Here we focus on explicit methods, for which the denominator is 1.

In [ ]:
def lower_triangular_solve(rk,SO=False): # Using lower_triangular_solve, which is always faster than LUsolve
    start = time.perf_counter()
    z = sympy.var('z')
    I = sympy.eye(len(rk))
    if SO:
        v = 1 - rk.alpha.sum(1)
        vstar = sympy.Matrix(v[:-1])
        v_mp1 = sympy.Rational(v[-1])
        alphastar=sympy.Matrix(rk.alpha[:-1,:])
        betastar=sympy.Matrix(rk.beta[:-1,:])
        alpha_mp1 = sympy.Matrix(rk.alpha[-1,:])
        beta_mp1 = sympy.Matrix(rk.beta[-1,:])
        p1=(alpha_mp1 + z*beta_mp1).transpose()*(I-alphastar-z*betastar).lower_triangular_solve(vstar)
        p1=p1[0].expand()+v_mp1
    else:
        Asym=sympy.Matrix(rk.A)
        bsym=sympy.Matrix(rk.b)
        e = sympy.ones(len(rk),1)
        p1=z*bsym.transpose()*(I-z*Asym).lower_triangular_solve(e)
        p1=p1[0].expand()+1
    p1=p1.as_poly(z).all_coeffs()
    p1=p1[::-1]
    q1=[sympy.Rational(1)]
    p=np.poly1d(p1[::-1])    # Numerator
    q=np.poly1d(q1[::-1])    # Denominator
    t = time.perf_counter()-start
    return p,q,t


def determinants(rk,SO=False): #Using charpoly
    start = time.perf_counter()
    z=sympy.var('z')
    if SO:
        I = sympy.eye(len(rk))
        v = 1 - rk.alpha.sum(1)
        vstar = sympy.Matrix(v[:-1]).T
        v_mp1 = sympy.Rational(v[-1])
        alphastar=sympy.Matrix(rk.alpha[:-1,:])
        betastar=sympy.Matrix(rk.beta[:-1,:])
        alpha_mp1 = sympy.Matrix(rk.alpha[-1,:])
        beta_mp1 = sympy.Matrix(rk.beta[-1,:])
        xsym = I - alphastar - z*betastar + vstar/v_mp1 * (alpha_mp1+z*beta_mp1)
        p1=v_mp1*xsym.charpoly(z).coeffs()
    else:
        Asym=sympy.Matrix(rk.A)
        bsym=sympy.Matrix(np.tile(rk.b,(len(rk),1)))
        xsym=Asym-bsym
        p1=xsym.charpoly(z).coeffs()

    q1=[sympy.Rational(1)]
    p=np.poly1d(p1[::-1])    # Numerator
    q=np.poly1d(q1[::-1])    # Denominator
    t = time.perf_counter()-start
    return p,q,t

def series(rk,SO=False): # Using power series
    start = time.perf_counter()
    s = rk.num_seq_dep_stages()
    z=sympy.var('z')
    I = sympy.eye(len(rk))
    if SO:
        alphastarsym = sympy.Matrix(rk.alpha[0:-1,:])
        betastarsym  = sympy.Matrix(rk.beta[0:-1,:])

        matsym = alphastarsym + betastarsym*z
        vecsym = sympy.Matrix(rk.alpha[-1,:]+z*rk.beta[-1,:])

    else:
        Asym=sympy.Matrix(rk.A)
        bsym=sympy.Matrix(rk.b)
        matsym = z*sympy.Matrix(rk.A)
        vecsym = z*sympy.Matrix(rk.b)

    # Compute (I-zA)^(-1) = I + zA + (zA)^2 + ... + (zA)^(s-1)
    matpow = I
    matsum = I
    for i in range(1,s):
        matpow = matpow*matsym
        matsum = matsum + matpow
    p1 = vecsym.transpose()*matsum
    if SO:
        v = 1 - rk.alpha.sum(1)
        vstar = sympy.Matrix(v[:-1])
        v_mp1 = sympy.Rational(v[-1])
        p1 = p1*vstar
        p1=p1[0].expand()+v_mp1
    else:
        e = sympy.ones(len(rk),1)
        p1 = p1*e
        p1=p1[0].expand()+1
    
    p1=p1.as_poly(z).all_coeffs()
    p1=p1[::-1]
    q1=[sympy.Rational(1)]
    p=np.poly1d(p1[::-1])    # Numerator
    q=np.poly1d(q1[::-1])    # Denominator
    t = time.perf_counter()-start
    return p,q,t

The following function just runs each of the above approaches on a given family of methods and plots the run-times for each approach:

In [ ]:
def time_stability_polynomial_calculation(method,pmin=2,pmax=5,approaches=[lower_triangular_solve,determinants,series],SO=False):
    norders = pmax-pmin+1
    orders = range(pmin,pmax+1)

    times = []
    for j in range(len(approaches)):
        times.append(np.zeros((norders,1)))

    for i, order in enumerate(orders):
        myrk = method(order)
        for j, approach in enumerate(approaches):
            p,q,t = approach(myrk,SO=SO)
            times[j][i] = t

    for j in range(len(approaches)):
        plt.plot(orders,times[j],linewidth=3)
        
    leg_text = [approach.__name__ for approach in approaches]
    plt.legend(leg_text,loc='best')
    plt.ylabel('time (s)')
    plt.xlabel('order')
    plt.title('Computation of stability polynomial for '+method.__name__)

In [ ]:
time_stability_polynomial_calculation(rk.extrap,pmax=7)

In [ ]:
time_stability_polynomial_calculation(rk.extrap,pmax=7,SO=True,approaches=[lower_triangular_solve,series])

Clearly the `lower_triangular_solve` method is the fastest for extrapolation methods.  The other two methods will quickly reach unreasonable run-times if we increase $p$ further.  Using the Shu-Osher form is noticeably faster for all methods.

In [ ]:
time_stability_polynomial_calculation(rk.DC,pmax=5)

In [ ]:
time_stability_polynomial_calculation(rk.DC,pmax=5, )

For deferred correction methods, we see roughly the opposite behavior: `lower_triangular_solve` is extremely slow compared to either of the other two.  Series has a slight edge over charpoly.  Again, using the Shu-Osher form is significantly faster for some methods.

In [ ]:
time_stability_polynomial_calculation(rk.RKC1,pmax=12)

In [ ]:
time_stability_polynomial_calculation(rk.RKC1,pmax=12,SO=True,approaches=[lower_triangular_solve,series])

For RKC methods, things are complicated.  The `determinants` approach is fastest if using the Butcher form, but the other two approaches are reasonably fast if using the Shu-Osher form (I really need to implement the Shu-Osher ration of determinants!) (I should add that the x-axis label here is wrong; it should really be "stages") 

In [ ]:
time_stability_polynomial_calculation(rk.SSPRK2,pmax=12)

In [ ]:
time_stability_polynomial_calculation(rk.SSPRK2,pmax=12,SO=True,approaches=[lower_triangular_solve,series])

SSPRK2 methods behave mostly like RKC methods, which is unsurprising since they have a very similar structure.  But they show much better results using `lower_triangular_solve` on the SO form.

So, we can implement all three options.  Which should be the default?  Let's try an "ordinary" Runge-Kutta method, not belonging to any of these families (but with a moderately large number of stages):

In [ ]:
approaches=[lower_triangular_solve,determinants,series]
SO=False
times = []
myrk = rk.loadRKM('PD8')
for j, approach in enumerate(approaches):
    p,q,t = approach(myrk,SO=SO)
    times.append(t)
list(zip([approach.__name__ for approach in approaches], times))

`lower_triangular_solve` is the clear loser.  So we'll go with `determinants` as the default if Butcher coefficients are used, `lower_triangular_solve` as the default if Shu-Osher arrays are used, but allow the user to pick any of the three.

For the implementation of `internal_stability_polynomials`, it seems not worthwhile to work out and implement a ratio of determinants formula, so we'll just provide the other two and default to Shu-Osher with `lower_triangular_solve`.